

__Set your own route planner for a given OSM map and GTFS file and query server on localhost for a detailed routes__


## OpenTripPlanner (OTP) walkthrough

1. Install Java JDK.
2. Download [latest](https://repo1.maven.org/maven2/org/opentripplanner/otp/1.4.0/otp-1.4.0-shaded.jar) .jar from [Maven](https://repo1.maven.org/maven2/org/opentripplanner/otp/1.4.0/).
3. Download gtfs.zip of a given city into a `folder` from `transit.land`.
4. Download osm.pbf of your area from [here](https://www.interline.io/osm/extracts/) (need to login to get API token).
5. Call from terminal `java -Xmx6G -jar otp-1.4.0-shaded.jar --build data_path --inMemory` changing `6G` as memory allocated (6GB) and `data_path` to your path with osm and gtfs files.
6. If succesful you shall be able to open a web search in your browser: [`http://localhost:8080/`](http://localhost:8080/).

information from [link](http://docs.opentripplanner.org/en/latest/Basic-Tutorial/)


In [1]:
import pandas as pd
import requests

### Local imports

In [2]:
from main import make_query, parse_OTP_response
from utils import plot

The `georequests.csv` file contains requests to travel from specific origin to destination points. 

In [3]:
PATH = 'georequests.csv'
OTP_API = "http://localhost:8080/otp/routers/default/plan"

The `origin_x`, `destination_x`, `origin_y` and `destination_y` represent the longitude (x) and latitude (y) coordinates of the origin and destination locations

In [4]:
df = pd.read_csv(PATH, index_col=[0])  # load the csv
df.treq = pd.to_datetime(df.treq)
df.sample(5)[['origin_x','origin_y','destination_x','destination_y','treq']]

,origin_x,origin_y,destination_x,destination_y,treq
647,4.875478,52.352987,4.886491,52.368780,2020-10-29 15:04:29
281,4.884157,52.347835,4.892685,52.365040,2020-10-29 14:40:07
638,4.888803,52.389063,4.894835,52.379590,2020-10-29 14:59:37
550,4.890601,52.341577,4.888396,52.370173,2020-10-29 14:45:09
394,4.882449,52.366186,4.891513,52.376213,2020-10-29 14:38:40


We generate a new DataFrame containing the details of our request, including the designated origin and destination for the journey.

In [5]:
df1 = pd.DataFrame(columns=['origin_x','origin_y','destination_x','destination_y','treq'])
df1['origin_x'] = [16.89423, 16.91294, 16.94984]
df1['origin_y'] = [52.40504, 52.42797, 52.41404]

df1['destination_x'] = [16.94984, 16.92324, 16.88410]
df1['destination_y'] = [52.40619, 52.39833, 52.36973]

df1['treq'] = pd.Timestamp(2023, 3, 7, 12)

df =df1
df

,origin_x,origin_y,destination_x,destination_y,treq
0,16.89423,52.40504,16.94984,52.40619,2023-03-07 12:00:00
1,16.91294,52.42797,16.92324,52.39833,2023-03-07 12:00:00
2,16.94984,52.41404,16.88410,52.36973,2023-03-07 12:00:00


### Sample query 

before you can query the server you need to run it -see `run OTP server.ipynb` in this folder

[query synthax](http://dev.opentripplanner.org/apidoc/1.0.0/resource_PlannerResource.html)  
[response json structure](http://dev.opentripplanner.org/apidoc/1.0.0/json_Response.html)

In [6]:
row = df.sample(1).squeeze() #randomly choose a row from df and convert it into a Pandas Series
query = make_query(row)
query

{'fromPlace': '52.42797,16.91294',
 'toPlace': '52.39833,16.92324',
 'time': '00:0pm',
 'date': '3-7-2023',
 'mode': 'TRANSIT,WALK',
 'maxWalkDistance': 2000,
 'arriveBy': 'false'}

The itinerary refers to a possible travel plan. 

In [7]:
r = requests.get(OTP_API, params=query)
route = parse_OTP_response(r.json())
route

{'success': True,
 'n_itineraries': 3,
 'duration': 1904,
 'walkDistance': 899.6474060001171,
 'transfers': 1,
 'transitTime': 1140,
 'waitingTime': 2,
 'modes': [['WALK', 400, 470],
  ['TRAM', 840, 3768],
  ['TRAM', 300, 1390],
  ['WALK', 362, 428]]}

#### Visualization

In [8]:
plot(r.json()['plan'], color = 'green')

Trip from (16.9129,52.4280) to (16.9232,52.3983) at 1678186800000. 
3 connections found. 
Best one is 32min (900m walk, 1 transfer(s), wait time 0.03min)

LEG 	 MODE 	DIST 	TIME
-----------------------------
1	WALK	470	400
2	TRAM	3768	840
3	TRAM	1390	300
4	WALK	429	362


#### Output

In [9]:
pd.read_csv('georequests_PT.csv', index_col=[0]).sample(5)

,success,n_itineraries,duration,walkDistance,transfers,transitTime,waitingTime,modes
id,,,,,,,,
592,True,3,601,435.817135,0,235,2,"[['WALK', 79, 92], ['TRAM', 235, 994], ['WALK'..."
475,True,3,995,845.790947,0,288,2,"[['WALK', 62, 73], ['TRAM', 288, 1314], ['WALK..."
550,True,3,1158,697.124383,0,324,242,"[['WALK', 188, 195], ['SUBWAY', 324, 3213], ['..."
568,True,3,1954,1195.218632,0,977,2,"[['WALK', 793, 982], ['TRAM', 977, 4072], ['WA..."
791,True,3,1588,1461.793601,0,420,2,"[['WALK', 891, 1110], ['BUS', 420, 2447], ['WA..."


#### Full response

In [10]:
r.json()['requestParameters']

{'date': '3-7-2023',
 'mode': 'TRANSIT,WALK',
 'arriveBy': 'false',
 'fromPlace': '52.42797,16.91294',
 'toPlace': '52.39833,16.92324',
 'time': '00:0pm',
 'maxWalkDistance': '2000'}

---